Test with Github

In [1]:
# Set up colab instance
from google.colab import drive
drive.mount('/content/drive/')




Mounted at /content/drive/


In [2]:
import os
os.chdir('drive/MyDrive/GIF-7005-Projet/gif-7005-projet/')

In [3]:
!pip3 install pydicom

     |████████████████████████████████| 1.9MB 5.7MB/s 


In [4]:
# Pull any change on the repo

!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/thomasp05/gif-705-projet
   0391015..c89f0de  main       -> origin/main
Updating 0391015..c89f0de
Fast-forward
 documentation/iaa-projet-a20.pdf | Bin 0 -> 82726 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 documentation/iaa-projet-a20.pdf


In [5]:
import time

import torch

from dataset import *
from models import *

torch.manual_seed(111)

In [6]:
# HYPERPARAMETERS
N_EPOCH = 10
BATCH_SIZE = 64

In [7]:
dataset = dcm_dataset('data')

train_set, test_set = train_test_split(dataset)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=BATCH_SIZE, num_workers=2)

In [8]:
cnn = Vanilla_CNN(width=16).to("cuda:0")
regressor = Simple_regressor(16 * 8 * 8).to("cuda:0")

In [9]:
params = list(cnn.parameters()) + list(regressor.parameters())
optim = torch.optim.Adam(params, lr=1e-4)

loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
cnn.train()
regressor.train()

for epoch in range(N_EPOCH):
    timer = time.time()

    img_processed = 0

    for img, target in train_loader:

        optim.zero_grad()

        img, target = img.to("cuda:0"), target.to("cuda:0")

        pred1 = cnn(img)
        pred2 = regressor(pred1.flatten(1, -1)).squeeze()

        loss = loss_fn(pred2, target.to(torch.float))

        loss.backward()

        optim.step()

        img_processed += img.shape[0]

    print("Epoch : {}".format(epoch + 1))
    print("Time elapsed : {}".format(time.time() - timer))

In [12]:
# Accuracy

cnn.eval()
regressor.eval()

Simple_regressor(
  (layer): Linear(in_features=1024, out_features=1, bias=True)
)

In [ ]:
# Accuracy

cnn.eval()
regressor.eval()

score = []

for img, target in test_loader:

    img, target = img.to("cuda:0"), target.to("cuda:0")

    pred1 = cnn(img)
    pred2 = regressor(pred1.flatten(1, -1)).squeeze()

    score_ = ((pred2 > 0.5) == (target == 1)).sum()

    score.append(score_)

score = torch.Tensor(score).sum() / len(test_loader.dataset)
print(score)